In [1]:
import pandas as pd
import json
import ast
import re
from constants import * 

In [ ]:
# создаем файл для категорий второго уровня
res = {i: dict(enumerate(values)) for i, values in enumerate(FIRST_LEVEL.values())}

with open('zeroshot/data/zeroshot_topics_2.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)

## Создание json

In [3]:
# очистка текста
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'Unknown node type: \w+', '', text)
    text = ' '.join(text.split())
    return text

# keywords в список
def parse_keywords(kw_str):
    if pd.isna(kw_str) or kw_str.strip() == "":
        return []
    try:
        return ast.literal_eval(kw_str)
    except Exception:
        return [kw_str]
    
# функция для создания правльного json
def prepare_json(df, filename, root, columns_rename=COLUMNS_RENAME):
    df_json = df[list(columns_rename.keys())].rename(columns=columns_rename)
    df_json['abstract'] = df_json['abstract'].apply(clean_text)
    df_json['keywords'] = df_json['keywords'].apply(parse_keywords)

    records = df_json.to_dict(orient='records')
    print(f'Размер {root}{filename} df: {df_json.shape[0]}')

    with open(f'{root}{filename}.json', 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=4)
    
    return f'{root}{filename}.json успешно сохранен' 

### 1 level

In [4]:
df = pd.read_csv('zeroshot/data/final_clean.csv')

df_train = df[df['first_OECD'] == 'нет'].copy()
df_test  = df[df['first_OECD'] != 'нет'].copy()

print(prepare_json(df_train, filename='train', root='zeroshot/data/train/'))
print(prepare_json(df_test, filename='test', root='zeroshot/data/test/'))

Размер zeroshot/data/train/train df: 2858
zeroshot/data/train/train.json успешно сохранен
Размер zeroshot/data/test/test df: 1303
zeroshot/data/test/test.json успешно сохранен


### 2 level

In [5]:
#df = pd.read_csv('')
df = pd.read_csv('zeroshot/data/final_clean.csv')
for i, k in enumerate(FIRST_LEVEL.keys()): 

    df_train = df[(df['first_OECD_pred'] == k) & (df['OECD'] == 'нет')].copy()
    df_test  = df[df['first_OECD'] == k].copy()

    print(prepare_json(df_train, filename=f'{i}', root='zeroshot/data/train/'))
    print(prepare_json(df_test, filename=f'{i}', root='zeroshot/data/test/'))

KeyError: 'first_OECD_pred'

In [11]:
import pandas as pd
df = pd.read_csv('zeroshot/data/final_clean.csv')
df[df['eLIBRARY ID'] == 49156026]

,Ссылка на статью,DOI,eLIBRARY ID,article_type,year,language,title_en,key_words_en,abstract_en,authors_metadata_en,...,authors_metadata_ru,OECD,Название журнала,Издательство,ISSN,eISSN,Цитирования,first_OECD,text,num_tokens
1566,https://elibrary.ru/item.asp?id=49156026,10.1007/s10958-022-05840-w,49156026,статья в журнале - научная статья,2022,английский,VICTOR TIMOFEEVICH MARKOV (21.06.1948–15.07.2019),[],Victor Timofeevich was a man of very broad vie...,"[{'Author': 'ADRIANOV N.M.', 'Institution': 'L...",...,NaN,Mathematics,JOURNAL OF MATHEMATICAL SCIENCES,Plenum Publishers,1072-3374,1573-8795,0.0,Natural Sciences,VICTOR TIMOFEEVICH MARKOV (21.06.1948–15.07.20...,96
